In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls /root/.kaggle/
!ls -l ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

kaggle.json
-rw-r--r-- 1 root root 67 Dec 27 09:13 /root/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d navoneel/brain-mri-images-for-brain-tumor-detection

brain-mri-images-for-brain-tumor-detection.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
from zipfile import ZipFile 
dataset = "/content/brain-mri-images-for-brain-tumor-detection.zip"
with ZipFile(dataset, 'r') as zip:
 zip.extractall()
 print('done')

done


In [ ]:
import cv2
import os
import tensorflow as tf
from tensorflow import keras 
from PIL import Image 
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import normalize
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.utils import to_categorical


image_Directory="DATASET/"
no_tumor_images=os.listdir(image_Directory+ '\no')
dataset=[]
label=[]
INPUT_SIZE=64

print(no_tumor_images)
for i, image_name in enumerate(no_tumor_images):
    if(image_name.split('.')[1]=="jpg"):
        image=cv2.imread(image_Directory +'no/'+image_name)
        image=Image.fromarray(image, "RGB")
        image=image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(0)

for i, image_name in enumerate(yes_tumor_images):
    if(image_name.split('.')[1]=="jpg"):
        image=cv2.imread(image_Directory +'yes/'+image_name)
        image=Image.fromarray(image, "RGB")
        image=image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(1)

dataset=np.array(dataset)
label=np.array(label)

x_train, x_test, y_train, y_test=train_test_split( dataset, label, test_size=0.2, rand_state=0)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)
x_train=normalize(x_train, axis=1)
x_test=normalize(x_test, axis=1)
y_train=to_categorical(y_train, num_classes=2)
y_test=to_categorical(y_test, num_classes=2)
model=Sequential()

model.add(Conv2D(32, (3,3), input_shape=(INPUT_SIZE, INPUT_SIZE, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train,
batch_size=16,
verbose=1, epochs=10,
validation_date=(x_test, y_test),
shuffle=False)

model.save('BrainTumor10Epochs.h5')

In [ ]:
from keras.models import load_model

model=load_model('BrainTumor10Epochs.h5')
image=cv2.imread('preDiction image path')

img=Image.fromarray(image)

img=img.resize((64,64))

img=np.array(img)
input_img=np.expand_dims(img, axis=0)

result=model.predict_classes(input_img)
print(result)